<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src='OCR_logo3.png'>

# Evaluating the Math Performance of NY State Middle Schools


_Version 1.0_  
_Author(s): Jon Reifschneider_

<img align="left" style="padding-top:10px;" src="NYSED_logo.png">

## _About this teaching case_
**Level:** Beginner  
**Language:** Python  
**Libraries:** Pandas, Matplotlib, Plotly  
**Industry:** Education

**Learning Topics:**  
- Data Manipulation
- Exploratory Data Analysis
- Geospatial Visualizations  

**Learning Objectives**   
- Build skills in merging, clearning and filtering data using pandas
- Learn to aggregate data and perform statistical and visual analyses
- Learn to visualize geospatial data using choropleths

**Pre-requisites**  
- Basic proficiency in Python and pandas

**Case Structure**  
This teaching case is structured to follow the ***modified CRISP-DM data science methodology*** used in Duke University's AI for Product Innovation graduate programs. 

**Datasets Used**  
NY State Department of Education 3-8 Assessment Database: https://data.nysed.gov/downloads.php

# Contents
[1: Business Understanding](#1)  
[2: Data Understanding](#2)  
[3: Data Preparation](#3)  
[4: Analysis / Modeling](#4)  
[5: Evaluation / Interpretation](#5)

In [ ]:
# This downloads the necessary data files into the same directory where you have saved this notebook
# Run this before any other code cell

import urllib.request
from pathlib import Path
import os
import zipfile
path = Path()

# Dictionary of file names and download links
files = {'NY_schools_data.zip':'https://storage.googleapis.com/aipi_datasets/NY_schools_data.zip'}

# Download file(s)
for key,value in files.items():
    filename = path/key
    url = value
    # Download and unzip if it does not already exist
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url,filename)
        zip_ref = zipfile.ZipFile(filename, 'r')
        zip_ref.extractall(path)
        zip_ref.close()

# Step 1: Business Understanding <a class="anchor" id="1"></a>
You have recently been engaged on a consulting assignment by the NY State Department of Education to identify ways to improve the math performance of middle school students across the state.  The DoE believes that NY middle school students are not performing at a competitive level to other states in the country.

The DoE has limited resources and would like to make data-driven decisions on how to deploy those resources to have maximum effect on the overall math performance of the state's middle school children.  One way to define success for our client by looking at the average increase in math scores as a result of the actions that we recommend, relative to historical year-over-year trends.

**Identify underperforming schools**   
One of the main factors under control of the DoE is where to allocate their annual budget, e.g. how to distribute it amongst the counties and schools in the state.  Our hypothesis is that by identifying the most grossly underperforming areas of the state and allocating more of the budget to those areas, we can maximize the impact of our dollars available to spend.  Our analysis today will focus on identifying the worst performing schools and counties in mathematics, in order to help the DoE make budget allocation decisions.

For our analysis, we have decided to define an "underperforming school" as one in which average math assessment scores for grade 8 students have been in the bottom 10% of scores across the state for each of the past three years.  Identifying underperforming schools can help us focus our state's investment and efforts towards improving math outcomes for students in those schools.

Being the brilliant data science consultant that you are, you know the next step is to look for data.  We know that in order to perform any useful analysis, we need data on average math assessment scores for grade 8 students broken down by school.

# Step 2: Data Understanding <a class="anchor" id="2"></a>
The NY State DoE maintains a database of aggregated assessment scores for grades 3-8 for each public middle school in the state dating back to 2013-14, broken down into various demographic groups.  We can use this data to analyze the last three years of historical data and identify underperforming schools in mathematics.  

The NY DoE data includes mean assessment scores broken down by demographic group. What if we have a situation where we have only 1 student from a certain demographic group in grade 8 at a school, and the mean assessment score for that group is obviously equal to that individual student's score.  Now suppose the school publishes a directory of its students, so we know who the student is.  Is this a violation of FERPA?

**Ethical considerations:**  
What if, rather than just analyzing the data for purposes of resource allocations, we were going to use our analysis to make staffing decisions - e.g. we were going to fire the math teachers at the bottom 10% of schools based on student assessment score last year.  Would this be considered ethical, according to the FAT (fair, accountable, transparent) standard?

What if we explained to all state teachers exactly how we were calculating things in our analysis, we used only scores (e.g. no demographic or other data), and we clarified for them exactly who to contact to discuss any possible errors in the data or calculations.  Would it then be considered ethical to fire the bottom 10%, based on the FAT standard?

## 2.1 Gather data

In [ ]:
# Run setup

import pandas as pd
import numpy as np
import plotly.figure_factory as ff

# Disable pandas warnings
pd.options.mode.chained_assignment = None  # default='warn'

# Read in the three data files, one for each year
datapath = 'NY_schools_data'
if not os.path.exists(datapath):
    raise FileNotFoundError(f'Expected data to be located in {os.path.abspath(path)}. Please get the files and try again.')
df_2017 = pd.read_csv(datapath+'/data_2017.csv')
df_2018 = pd.read_csv(datapath+'/data_2018.csv')
df_2019 = pd.read_csv(datapath+'/data_2019.csv')

One issue we can see with the raw data is that we do not have a unique identifier for each school.  There may be multiple schools within the state that have the same name (e.g. it turns out there are 26 schools in New York with the name "Park Avenue School").  So let's create a new column in each of our raw data dataframes which has a unique identifier for each school

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

Then, merge the data from each year into a single dataframe

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

## 2.2 Validate the data

Now that we have our data organized in a dataframe, let's clean up the data:
- Remove rows with no scores
- Remove rows containing average scores across state/regions
- Simplify the columns by removing unnecessary or duplicative columns
- Convert column types as needed

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

# Step 3: Data Preparation <a class="anchor" id="3"></a>

### Filter the data

Filter the dataset to get only Grade 8 math scores

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

When we look at the mean scores, we see a potential issue - it looks like scores for 2017 are about half of the scores for 2018/19.  Let's check if this is true.  If so, we should put our scores on a common scale for our analysis.  First, confirm that this is true

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

Use min-max scaling to put the scores from each year onto a common 0-1 scale

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

# Step 4: Analysis / Modeling <a class="anchor" id="4"></a>

Now that we have our data cleaned and filtered, it's time to begin our analysis.  Let's start with the first part of our assignment to identify the underperforming schools.  As a reminder, we have defined the "underperforming schools" as those schools whose score for each of the 3 years falls in the bottom 10% of all schools.  Filter your data to create a new dataframe which includes only the underperforming schools.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

# Step 5: Evaluation & Interpretation <a class="anchor" id="5"></a>

How many underperforming schools do we have? What was their mean scaled Grade 8 math score across the 3 years?

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

How do the underperforming schools vary by geography (county)?

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

### Plotting the underperformers by county

Let's create a visualization for our client of where the underperforming schools are.  Our visualization will plot the concentration of underperforming schools by county. To assist with this, the following cell reads in the table of FIPS codes which is needed to plot the counties correctly.

In [ ]:
# First we need to map the county names to FIPS codes for plotting on a geographic map.  
# We can use the US Census FIPS code list availabe at 
# https://www.census.gov/geographies/reference-files/2016/demo/popest/2016-fips.html to do this

fips_codes = pd.read_excel(datapath+'/all-geocodes-v2016.xlsx',header=4)
fips_codes.rename({'Area Name (including legal/statistical area description)':'Area Reference'},axis=1,inplace=True)

Match the county names where are underperforming schools are located to the FIPS codes from our table, and then use plotly to create a choropleth plot of underperforming schools by county for NY state.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION